# Import Car Economy Data

In [88]:
import pandas as pd
from pandas import DataFrame 
df_tennis = pd.read_csv("EconomyCar.csv")
print("\n Given Car Eco Data Set:\n\n", df_tennis.head(10))


 Given Car Eco Data Set:

   Country     Brand  Color  Year     Type  Eco
0   Japan     Honda   Blue  1980  Economy  Yes
1   Japan    Toyota  Green  1970   Sports   No
2     USA    Toyota   Blue  1990  Economy  Yes
3     USA  Chrysler    Red  1980  Economy   No
4   Japan  Chrysler  White  1970   Sports  Yes
5     USA     Honda    Red  1990  Economy  Yes
6   Japan     Honda  White  1980  Economy   No
7   Japan    Toyota  Green  1980   Sports  Yes


# Entropy of the Data Set


In [89]:
#Function to calculate the entropy of probaility of observations
# -p*log2*p

def entropy(probs):  
    import math
    return sum( [-prob*math.log(prob, 2) for prob in probs] )

#Function to calulate the entropy of the given Data Sets/List with respect to target attributes
def entropy_of_list(a_list):  
    #print("A-list",a_list)
    from collections import Counter
    cnt = Counter(x for x in a_list)   # Counter calculates the propotion of class
   # print("\nClasses:",cnt)
    #print("No and Yes Classes:",a_list.name,cnt)
    num_instances = len(a_list)*1.0   # = 8
    print("\n Number of Instances of the Current Sub Class is {0}:".format(num_instances ))
    probs = [x / num_instances for x in cnt.values()]  # x means no of YES/NO
    print("\n Classes:",min(cnt),max(cnt))
    print(" \n Probabilities of Class {0} is {1}:".format(min(cnt),min(probs)))
    print(" \n Probabilities of Class {0} is {1}:".format(max(cnt),max(probs)))
    return entropy(probs) # Call Entropy :
    
# The initial entropy of the YES/NO attribute for our dataset.
print("\n  INPUT DATA SET FOR ENTROPY CALCULATION:\n", df_tennis['Eco'])

total_entropy = entropy_of_list(df_tennis['Eco'])

print("\n Total Entropy of Car Economy Data Set:",total_entropy)


  INPUT DATA SET FOR ENTROPY CALCULATION:
 0    Yes
1     No
2    Yes
3     No
4    Yes
5    Yes
6     No
7    Yes
Name: Eco, dtype: object

 Number of Instances of the Current Sub Class is 8.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.375:
 
 Probabilities of Class Yes is 0.625:

 Total Entropy of Car Economy Data Set: 0.9544340029249649


# Information Gain of Attributes 

In [90]:
def information_gain(df, split_attribute_name, target_attribute_name, trace=0):
    print("Information Gain Calculation of ",split_attribute_name)
    '''
    Takes a DataFrame of attributes, and quantifies the entropy of a target
    attribute after performing a split along the values of another attribute.
    '''
    # Split Data by Possible Vals of Attribute:
    df_split = df.groupby(split_attribute_name)
   # for name,group in df_split:
    #    print("Name:\n",name)
     #   print("Group:\n",group)
    
    # Calculate Entropy for Target Attribute, as well as
    # Proportion of Obs in Each Data-Split
    nobs = len(df.index) * 1.0
   # print("NOBS",nobs)
    df_agg_ent = df_split.agg({target_attribute_name : [entropy_of_list, lambda x: len(x)/nobs] })[target_attribute_name]
    #print([target_attribute_name])
    #print(" Entropy List ",entropy_of_list)
    #print("DFAGGENT",df_agg_ent)
    df_agg_ent.columns = ['Entropy', 'PropObservations']
    #if trace: # helps understand what fxn is doing:
     #   print(df_agg_ent)
    
    # Calculate Information Gain:
    new_entropy = sum( df_agg_ent['Entropy'] * df_agg_ent['PropObservations'] )
    old_entropy = entropy_of_list(df[target_attribute_name])
    return old_entropy - new_entropy


print('Info-gain for Country is :'+str( information_gain(df_tennis, 'Country', 'Eco')),"\n")
print('\n Info-gain for Brand is: ' + str( information_gain(df_tennis, 'Brand', 'Eco')),"\n")
print('\n Info-gain for Color is:' + str( information_gain(df_tennis, 'Color', 'Eco')),"\n")
print('\n Info-gain for Year is:' + str( information_gain(df_tennis, 'Year','Eco')),"\n")
print('\n Info-gain for Type is:' + str( information_gain(df_tennis, 'Type','Eco')),"\n")


Information Gain Calculation of  Country

 Number of Instances of the Current Sub Class is 5.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.4:
 
 Probabilities of Class Yes is 0.6:

 Number of Instances of the Current Sub Class is 3.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.3333333333333333:
 
 Probabilities of Class Yes is 0.6666666666666666:

 Number of Instances of the Current Sub Class is 8.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.375:
 
 Probabilities of Class Yes is 0.625:
Info-gain for Country is :0.0032289436203634114 

Information Gain Calculation of  Brand

 Number of Instances of the Current Sub Class is 2.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.5:
 
 Probabilities of Class Yes is 0.5:

 Number of Instances of the Current Sub Class is 3.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.3333333333333333:
 
 Probabilities of Class Yes is 0.6666666666666666:

 Number of Instances of the Current Sub Class is 3.0:

 Classes: 

# ID3 Algorithm

In [91]:
def id3(df, target_attribute_name, attribute_names, default_class=None):
    
    ## Tally target attribute:
    from collections import Counter
    cnt = Counter(x for x in df[target_attribute_name])# class of YES /NO
    
    ## First check: Is this split of the dataset homogeneous?
    if len(cnt) == 1:
        return next(iter(cnt))  # next input data set, or raises StopIteration when EOF is hit.
    
    ## Second check: Is this split of the dataset empty?
    # if yes, return a default value
    elif df.empty or (not attribute_names):
        return default_class  # Return None for Empty Data Set
    
    ## Otherwise: This dataset is ready to be devied up!
    else:
        # Get Default Value for next recursive call of this function:
        default_class = max(cnt.keys()) #No of YES and NO Class
        # Compute the Information Gain of the attributes:
        gainz = [information_gain(df, attr, target_attribute_name) for attr in attribute_names] #
        index_of_max = gainz.index(max(gainz)) # Index of Best Attribute
        # Choose Best Attribute to split on:
        best_attr = attribute_names[index_of_max]
        
        # Create an empty tree, to be populated in a moment
        tree = {best_attr:{}} # Iniiate the tree with best attribute as a node 
        remaining_attribute_names = [i for i in attribute_names if i != best_attr]
        
        # Split dataset
        # On each split, recursively call this algorithm.
        # populate the empty tree with subtrees, which
        # are the result of the recursive call
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3(data_subset,
                        target_attribute_name,
                        remaining_attribute_names,
                        default_class)
            tree[best_attr][attr_val] = subtree
        return tree

# Predicting Attributes

In [92]:
# Get Predictor Names (all but 'class')
attribute_names = list(df_tennis.columns)
print("List of Attributes:", attribute_names) 
attribute_names.remove('Eco') #Remove the class attribute 
print("Predicting Attributes:", attribute_names)

List of Attributes: ['Country', 'Brand', 'Color', 'Year', 'Type', 'Eco']
Predicting Attributes: ['Country', 'Brand', 'Color', 'Year', 'Type']


In [93]:
# Run Algorithm:
from pprint import pprint
tree = id3(df_tennis,'Eco',attribute_names)
print("\n\nThe Resultant Decision Tree is :\n")
#print(tree)
pprint(tree)
attribute = next(iter(tree))
print("Best Attribute :\n",attribute)
print("Tree Keys:\n",tree[attribute].keys())

Information Gain Calculation of  Country

 Number of Instances of the Current Sub Class is 5.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.4:
 
 Probabilities of Class Yes is 0.6:

 Number of Instances of the Current Sub Class is 3.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.3333333333333333:
 
 Probabilities of Class Yes is 0.6666666666666666:

 Number of Instances of the Current Sub Class is 8.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.375:
 
 Probabilities of Class Yes is 0.625:
Information Gain Calculation of  Brand

 Number of Instances of the Current Sub Class is 2.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.5:
 
 Probabilities of Class Yes is 0.5:

 Number of Instances of the Current Sub Class is 3.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.3333333333333333:
 
 Probabilities of Class Yes is 0.6666666666666666:

 Number of Instances of the Current Sub Class is 3.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.333333333